In [8]:
import os, sys
root_dir = os.path.dirname(os.getcwd())
sys.path.append(root_dir)

In [10]:
from configs import task_dataframe

In [1]:
import matplotlib as mpl
import pandas as pd
import numpy as np
import scipy.stats, os, json
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict

In [9]:
df = pd.read_pickle(f'{root_dir}/results/results.pkl')

In [3]:
df

,_wandb,detail,visual,obj,_step,overall,textual,_runtime,_timestamp,valid_count,...,model,stage,task_id,x_space,file_type,task_type,misleading,theta_space,run_name,run_id
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,gpt4v,eval,3,background,text,background_animal,1,animal,treasured-pyramid-522,pnat4yym
1,{'runtime': 254},266.0,266.0,339.0,0.0,5.0,339.0,255.654619,1.704788e+09,1000.0,...,gpt4v,eval,2,object,text,color_object,1,color,worthy-shadow-521,c6eratxz
2,{'runtime': 241},172.0,530.0,530.0,0.0,17.0,172.0,241.766935,1.704788e+09,1000.0,...,gpt4v,eval,1,color,text,color_object,1,object,scarlet-haze-520,vx2m12he
3,{'runtime': 258},604.0,604.0,10.0,0.0,4.0,10.0,260.177528,1.704787e+09,1000.0,...,emu,eval,10,object,text,texture_object,1,texture,vague-oath-519,4zbd5meq
4,{'runtime': 250},892.0,892.0,36.0,0.0,34.0,36.0,251.662383,1.704787e+09,1000.0,...,emu,eval,10,object,text,texture_object,0,texture,morning-microwave-518,2eaa07n9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
508,{'runtime': 265},732.0,827.0,827.0,0.0,571.0,732.0,266.444544,1.704498e+09,1000.0,...,qwen,eval,5,style,text,style_object,0,object,dulcet-sea-7,oca9fzwp
509,{'runtime': 257},376.0,376.0,981.0,0.0,364.0,981.0,258.766700,1.704498e+09,1000.0,...,qwen,eval,4,animal,text,background_animal,0,background,rare-wildflower-6,dwmc0azq
510,{'runtime': 275},759.0,626.0,626.0,0.0,459.0,759.0,276.669658,1.704498e+09,1000.0,...,qwen,eval,3,background,text,background_animal,0,animal,effortless-sea-5,txy4h3x0
511,{'runtime': 262},722.0,722.0,828.0,0.0,568.0,828.0,264.173308,1.704497e+09,1000.0,...,qwen,eval,2,object,text,color_object,0,color,still-river-4,ec4ln2nd


In [16]:
def get_acc(model, mode, shot, task_id, metric, misleading):
    run_df = df[
        (df['model'] == model) &
        (df['file_type'] == mode) &
        (df['shot'] == shot) &
        (df['task_id'] == task_id) &
        (df['misleading'] == misleading)
    ]
    
    if len(run_df):
        return run_df[metric].item()/1000
    else:
        return 0

In [20]:
get_acc(
    model = 'gpt4v',
    mode = 'text',
    shot = 2,
    task_id = 1,
    metric = 'overall',
    misleading = 0,
)

0.844

In [15]:
model_dict = {
    'image': {
        'emu': 'Emu',
        'gill': 'GILL',
        'seed': 'SEED-LLaMA',
        'emu2': 'Emu2',
    },
    'text': {
        'emu': 'Emu',
        'gill': 'GILL',
        'seed': 'SEED-LLaMA',
        'emu2': 'Emu2',
        'gpt4v': 'GPT-4V',
        'qwen': 'Qwen-VL',
        'llava': 'LLaVA',
    }
}

colors = ['#073B4C','#FFD166','#06D6A0','#118AB2', '#DD3497', '#AE017E', '#7A0177', '#49006A' ]
markers = ['o', '<', 's', 'p', 'P', 'X', 'D', 'd', 'v', 'h', 'H', '8', '>', '*', '1', '2', '3', '4', 'x', '+', '|', '_']
plt.rc('font', family='serif', serif='times new roman')
plt.rc('text', usetex=True)
plt.rc('xtick', labelsize=20)
plt.rc('ytick', labelsize=20)
mpl.rcParams['patch.linewidth']=0.5 #width of the boundary of legend

In [31]:
def get_shot_bars(
    num_shot,
    task_inference,
    model_mode,
    metric,
    misleading,
):
    task_df = pd.DataFrame(task_dataframe).T
    results = []
    if task_inference == 'object':
        tasks = task_df[(task_df.theta_space == 'object') | (task_df.theta_space == 'animal')]
    else:
        tasks = task_df[(task_df.theta_space != 'object') & (task_df.theta_space != 'animal')]
    for task_id in tasks.index:
        for model_label in model_dict[model_mode]:
            acc = get_acc(
                model = model_label,
                mode = model_mode,
                shot = num_shot,
                task_id = task_id,
                metric = metric,
                misleading = misleading,
            )
            results.append({
                'acc': acc,
                'model': model_label,
                'task_id': task_id,
                'task': task_dataframe[task_id]['task_name']
            })
            
    results_df = pd.DataFrame(results)
    return results_df


In [26]:
get_shot_bars(
    num_shot = 2,
    task_inference ='object',
    model_mode = 'text',
    metric = 'overall',
    misleading = 0,
)

,acc,model,task_id
0,0.146,emu,1
1,0.077,gill,1
2,0.129,seed,1
3,0.000,emu2,1
4,0.844,gpt4v,1
5,0.475,qwen,1
6,0.049,llava,1
7,0.404,emu,3
8,0.035,gill,3
9,0.019,seed,3


In [33]:
bar_width = 2.5
task_inference = 'object'
model_mode = 'text'
width, height = 15, 5
misleading = 1
metric = 'overall'
wspace = 6
barspace = 11

r1_data = get_shot_bars(
    num_shot = 2,
    task_inference = task_inference,
    model_mode = model_mode,
    metric = metric,
    misleading = misleading,
)
r1_data = [{'position': barspace*i + wspace*(i//len(model_dict[model_mode])), **r1_data.iloc[i]} for i in range(len(r1_data))]


r2_data = get_shot_bars(
    num_shot = 4,
    task_inference = task_inference,
    model_mode = model_mode,
    metric = metric,
    misleading = misleading,
)
r2_data = [{'position': barspace*i + wspace*(i//len(model_dict[model_mode])) + bar_width, **r2_data.iloc[i]} for i in range(len(r2_data))]

r3_data = get_shot_bars(
    num_shot = 6,
    task_inference = task_inference,
    model_mode = model_mode,
    metric = metric,
    misleading = misleading,
)
r3_data = [{'position': barspace*i + wspace*(i//len(model_dict[model_mode])) + 2 * bar_width, **r3_data.iloc[i]} for i in range(len(r3_data))]

r4_data = get_shot_bars(
    num_shot = 8,
    task_inference = task_inference,
    model_mode = model_mode,
    metric = metric,
    misleading = misleading,
)
r4_data = [{'position': barspace*i + wspace*(i//len(model_dict[model_mode])) + 3 * bar_width, **r4_data.iloc[i]} for i in range(len(r4_data))]


fig, ax = plt.subplots(nrows = 1, ncols = 1)
fig.subplots_adjust(left=0.05, bottom=.45, right=0.98, top=.88, wspace=0, hspace = 0.3) #margin of the figure
fig.set_size_inches(width, height) 

xticks, xticklabels = [], []
for i, (v1, v2, v3) in enumerate(zip(r1_data, r2_data, r3_data)):
    ax.bar(v1['position'], v1['acc'], width = bar_width, color = colors[0], label = '1-shot')
    ax.bar(v2['position'], v2['acc'], width = bar_width, color = colors[1], label = '2-shot')
    ax.bar(v3['position'], v3['acc'], width = bar_width, color = colors[2], label = '4-shot')
    xticks.append(v2['position'])
    xticklabels.append(v2['model'])
    
    if i % len(model_dict[model_mode]) == (len(model_dict[model_mode])-1)//2:
        ax.text(v2['position'], 0.85, v2['task'], fontsize=22, fontweight='bold', ha='center')
    
    # add legend
    if i == 0:
        ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.33), ncol=3, fontsize=22)
        
ax.set_xticks(xticks)
ax.set_xlim(-wspace, wspace + barspace*(len(r1_data)-1) + wspace*(len(r1_data)//len(model_dict[model_mode])))
# break the line of the xticklabels
ax.set_xticklabels(xticklabels, rotation = 90)
ax.set_yticks(np.arange(0, 1.2, 0.2))
# annotate every three clusters of bars with the dataset name
fig.supxlabel('Overall Generation Accuracy', fontsize=22)
save_path = f'{root_dir}/figures/{task_inference}_inference_m_{misleading}_{model_mode}.pdf'
os.makrdirs(save_path, exist_ok=True)
fig.savefig(save_path)
plt.show()

AttributeError: module 'os' has no attribute 'makrdirs'

Error in callback <function _draw_all_if_interactive at 0x7f095e83ae50> (for post_execute):


RuntimeError: latex was not able to process the following string:
b'lp'

Here is the full command invocation and its output:

latex -interaction=nonstopmode --halt-on-error --output-directory=tmpb0loc_s1 b1f6e526c8f59347b3e60c011cbf0b53.tex

This is pdfTeX, Version 3.14159265-2.6-1.40.20 (TeX Live 2019/Debian) (preloaded format=latex)
 restricted \write18 enabled.
entering extended mode
(./b1f6e526c8f59347b3e60c011cbf0b53.tex
LaTeX2e <2020-02-02> patch level 2
L3 programming layer <2020-02-14>
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2019/12/20 v1.4l Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))
(/usr/share/texlive/texmf-dist/tex/latex/type1cm/type1cm.sty)

! LaTeX Error: File `type1ec.sty' not found.

Type X to quit or <RETURN> to proceed,
or enter new name. (Default extension: sty)

Enter file name: 
! Emergency stop.
<read *> 
         
l.9 \usepackage
               [utf8]{inputenc}^^M
No pages of o

RuntimeError: latex was not able to process the following string:
b'lp'

Here is the full command invocation and its output:

latex -interaction=nonstopmode --halt-on-error --output-directory=tmp5nu0vnjm b1f6e526c8f59347b3e60c011cbf0b53.tex

This is pdfTeX, Version 3.14159265-2.6-1.40.20 (TeX Live 2019/Debian) (preloaded format=latex)
 restricted \write18 enabled.
entering extended mode
(./b1f6e526c8f59347b3e60c011cbf0b53.tex
LaTeX2e <2020-02-02> patch level 2
L3 programming layer <2020-02-14>
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2019/12/20 v1.4l Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))
(/usr/share/texlive/texmf-dist/tex/latex/type1cm/type1cm.sty)

! LaTeX Error: File `type1ec.sty' not found.

Type X to quit or <RETURN> to proceed,
or enter new name. (Default extension: sty)

Enter file name: 
! Emergency stop.
<read *> 
         
l.9 \usepackage
               [utf8]{inputenc}^^M
No pages of o

<Figure size 1500x500 with 1 Axes>